# Lab 7 |&nbsp;&nbsp;The knights of the "square" table

 &copy;&nbsp; murina 2020 

Today is the annual knigth meeting in the Royal Palace of Madrid. There are a square table and s seats for p<s persons. All the people must to keep distance in order to stop the spread of the Covid-19, a pandemic disease that affected the world at the end of 2019.<br>
Describe the situation as a computational problem and encode it in a QUBO representation. To make things easier, only think of the case of an even number of knights.

### Solution

<br>
<br>

#### Encoding the scenario in bits


bit => seat<br>
|0| => seat: available<br>
|1| => seat: occupied<br>
<br>
So, we are using a QUBO representation for the computational problem.<br>
<br>
Example with p=6 and s=8: 
<br>
<br>
<br>

## <ins>>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;0&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;0&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>&emsp;<ins>&nbsp;1&nbsp;</ins>

<br>
<br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; |0|&emsp;&emsp;&emsp;|1|<br>
&emsp;&emsp;&emsp;&emsp;x &nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x<br>
&emsp;&emsp;&emsp;&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x<br>
&emsp;&emsp;|1|&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x&emsp;|1|<br>
&emsp;&emsp;&emsp;&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x<br>
&emsp;&emsp;|1|&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x&emsp;|1|<br>
&emsp;&emsp;&emsp;&emsp;x&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;x<br>
&emsp;&emsp;&emsp;&emsp;x &nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x&nbsp;x<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; >|1|&emsp;&emsp;&emsp;|0|<br>
<br>
<br>

#### Encoding the computational problem as an interaction model in QUBO representation

For the meeting we need all the knights sitting:
<br>
<br>
1) all the people sitting:
<br>
$$(\sum_{i} x_{i}-p)^{2} = 0$$

<br>
<br>
<br>
$$=(1-2p)\sum_{i} x_{i}+2\sum_{<i,j>}x_{i}x_{j}+p^{2}$$
<br>
<br>
In order to keep distance, we should avoid:<br>
<br>
<br>
2) people sitting on both sides of each knight<br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\textrm{addition_of_the_ penalty}  \hspace{2cm} \sum_{i}      m{\hspace{0.1cm}}x_{i}x_{i+1} \hspace{2cm} m\in{\mathbb {R}>0} $
<br>
<br>
<br>
<br>
3) people sitting in front of each knight <br>
<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\textrm{addition_of_the_ penalty} \hspace{2cm}\sum_{i}  l{\hspace{0.1cm}}x_{i}x_{i+j(i)} \hspace{2cm} l\in{\mathbb {R}>0} \hspace{2cm}$<br>
<br>

$$j(i) \hspace{0.3cm}\textrm{is_the_index_of_the_seat_in_front_of_seat_i}$$

<br>
<br>


#### Tuning the Lagrance multipliers $\lambda_{i}$

<br>

## $$\lambda_{0}(\sum_{i} x_{i}-2p)^{2} + \lambda_{1}\sum_{i}m{\hspace{0.1cm}}x_{i}x_{i+1}+ \lambda_{2}\sum_{i}l{\hspace{0.1cm}}x_{i}x_{i+j(i)}= 0$$



### 8 seats for 4 knights 

####  QUBO scalar representation (coding h and J)
<br>

In [ ]:
import dimod   # is a shared API for binary quadratic samplers
               # provides a binary quadratic model class that contains Ising and QUBO models

from collections import defaultdict   # the input for the sampler will be constructed 
                                      # from a Python dictionary


# It is assumed m and l equal to 1, so it is no necessary to code them     
    
    
seats = 8
persons = 4

h = defaultdict(int)

J = defaultdict(int)

lambda_0 = 1


e_offset = lambda_0 * persons**2

for _ in range(seats):
    
    h[ _ ] = lambda_0 * ( 1 - 2*persons )   

for _ in range(seats - 1):
    for __ in range( _ + 1, seats):
        J[ _ , __ ] =  lambda_0*2    
    

lambda_1 = 1   
    
for _ in range(seats-1):
    J[ _ , _ + 1] += lambda_1
    
J[0, seats - 1] += lambda_1 
    
#####################################################################################################
# PAY ATTENTION: with this tuning, the result will be correct even we have not code the condition 3)      
####################################################################################################

bqm = dimod.BinaryQuadraticModel(h,J,
                                 e_offset,
                                 'BINARY')   #dimod.BINARY)

results = dimod.ExactSolver().sample(bqm)



In [ ]:
smpl = results.first.sample


print(smpl)
print("           ",smpl[7],"   ",smpl[6])
print("          * * * * * * ")
print("       *               *")
print(smpl[0],"     *               *  ",smpl[5])
print("       *               *")
print(smpl[1],"     *               *  ",smpl[4])
print("       *               *")
print("          * * * * * *  ")
print("           ",smpl[2],"   ",smpl[3])

####  QUBO matrix representation
<br>

In [ ]:
import dimod

from collections import defaultdict


seats = 8
persons = 4

Q = defaultdict(int)

lambda_0 = 1 


e_offset = lambda_0 * persons**2

for _ in range(seats):
    
    Q[ _ , _ ] = lambda_0 * ( 1 - 2*persons )   

for _ in range(seats - 1):
    for __ in range( _ + 1, seats):
        Q[ _ , __ ] =  lambda_0*2    
    

lambda_1 = 1   
    
for _ in range(seats-1):
    Q[ _ , _ + 1] += lambda_1
    
Q[0, seats - 1] += lambda_1 
    

#####################################################################################################
# PAY ATTENTION: with this tuning, the result will be correct even we have not code the condition 3)      
####################################################################################################
        

# we transform the dictionary into a bqm object as input of the sampler

bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset=e_offset)

results = dimod.ExactSolver().sample(bqm)

In [ ]:
smpl = results.first.sample

print(smpl)
print("           ",smpl[7],"   ",smpl[6])
print("          * * * * * * ")
print("       *               *")
print(smpl[0],"     *               *  ",smpl[5])
print("       *               *")
print(smpl[1],"     *               *  ",smpl[4])
print("       *               *")
print("          * * * * * *  ")
print("           ",smpl[2],"   ",smpl[3])

 ### 12 seats for 6 knights

#### QUBO matrix representation

In [ ]:
import dimod

from collections import defaultdict


seats = 12
persons = 6

J = defaultdict(int)


#####################################################################################################
# PAY ATTENTION: now, we DO apply all the conditions 1), 2), and 3) to optain a correct output      
##################################################################################################

lambda_0 = 1.2 


e_offset = lambda_0 * persons**2

for _ in range(seats):
    
    J[ _ , _ ] = lambda_0 * ( 1 - 2*persons )   

for _ in range(seats - 1):
    for __ in range( _ + 1, seats):
        J[ _ , __ ] =  lambda_0*2    
    

lambda_1 = 1   
    
for _ in range(seats-1):
    J[ _ , _ + 1] += lambda_1
    
J[0, seats - 1] += lambda_1 
    
    
lambda_2 = 1
    
J[0,8] += lambda_2 
J[1,7] += lambda_2    
J[2,6] += lambda_2    
J[3,11] += lambda_2
J[4,10] += lambda_2
J[5,9] += lambda_2


bqm = dimod.BinaryQuadraticModel.from_qubo(J, offset= e_offset)

results = dimod.ExactSolver().sample(bqm)

In [ ]:
smpl = results.first.sample

print(smpl)
print("         ",smpl[11] , "   ", smpl[10], "   ", smpl[9])
print("         * * * * * * * * *   ")

print("       *                   *")
print(smpl[0],"     *                   *  ",smpl[8])
print("       *                   *")
print(smpl[1],"     *                   *  ",smpl[7])
print("       *                   *")
print(smpl[2],"     *                   *  ",smpl[6])
print("       *                   *")
print("         * * * * * * * * *   ")
print("         ",smpl[3],"   ",smpl[4], "   ", smpl[5])